In [2]:
import pandas as pd
import io

# Import RDS password
#config = "/content/config.py"
#!pip install config
#import config
#from config import rds_pwd

import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [55.5 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-07 11:25:30--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.79MB/s    in 0.2s    

2021-05-07 11:25:31 (4.79 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Wine-Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

**Load Data into Spark DataFrame**

In [5]:
from pyspark import SparkFiles
url = "https://green-team-wine.s3.us-east-2.amazonaws.com/winemag-data-130k-v2.csv"
spark.sparkContext.addFile(url)
wine_df = spark.read.csv(SparkFiles.get("winemag-data-130k-v2.csv"), sep=",", header=True, inferSchema=True)
wine_df.show()

+---+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
|_c0|  country|         description|         designation|points|price|         province|           region_1|         region_2|       taster_name|taster_twitter_handle|               title|           variety|             winery|
+---+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
|  0|    Italy|Aromas include tr...|        Vulkà Bianco|    87| null|Sicily & Sardinia|               Etna|             null|     Kerin O’Keefe|         @kerinokeefe|Nicosia 2013 Vulk...|       White Blend|            Nicosia|
|  1| Portugal|This is ripe and ...|            Avidagos|    87| 15.0|            Douro|

In [6]:
wine_df.count()

129975

In [7]:
url = "https://green-team-wine.s3.us-east-2.amazonaws.com/winery_weather_subset.csv"
spark.sparkContext.addFile(url)
weather_df = spark.read.csv(SparkFiles.get("winery_weather_subset.csv"), sep=",", header=True, inferSchema=True)
weather_df.show()

+---+--------------------+----------------------------+--------------------------+--------------------+--------------------------+
|_c0|              Winery|Average Temperature (Kelvin)|Average Air Pressure (hPa)|Average Humidity (%)|Average Precipitation (mm)|
+---+--------------------+----------------------------+--------------------------+--------------------+--------------------------+
|  0|St. Julian winery US|                    269.6375|                   1018.15|               79.24|                    0.0725|
|  1|Sweet Cheeks wine...|                     276.025|                 1019.8875|             85.4525|                    0.0575|
|  2|Kirkland Signatur...|                     277.375|                 1020.0125|             79.1975|                     0.235|
|  3|Louis M. Martini ...|                    281.0325|                    1018.5|              79.155|                    0.0775|
|  4|Richard Böcking w...|                      275.71|                 1014.9425| 

In [8]:
weather_df.count()

4534

**Create DataFrames to match tables**

In [15]:
#Select only the columns that will be stored into the RDS database

from pyspark.sql.functions import col

wine_table_df = wine_df.select(col("country").alias("country"), col("points").alias("points"), col("province").alias("province"), col("region_1").alias("region_1"), col("region_2").alias("region_2"), col("title").alias("title"), col("variety").alias("variety"), col("winery").alias("winery")).where(col("country").isNotNull())
wine_table_df.show()


+---------+------+-----------------+-------------------+-----------------+--------------------+------------------+-------------------+
|  country|points|         province|           region_1|         region_2|               title|           variety|             winery|
+---------+------+-----------------+-------------------+-----------------+--------------------+------------------+-------------------+
|    Italy|    87|Sicily & Sardinia|               Etna|             null|Nicosia 2013 Vulk...|       White Blend|            Nicosia|
| Portugal|    87|            Douro|               null|             null|Quinta dos Avidag...|    Portuguese Red|Quinta dos Avidagos|
|       US|    87|           Oregon|  Willamette Valley|Willamette Valley|Rainstorm 2013 Pi...|        Pinot Gris|          Rainstorm|
|       US|    87|         Michigan|Lake Michigan Shore|             null|St. Julian 2013 R...|          Riesling|         St. Julian|
|       US|    87|           Oregon|  Willamette Valley

In [16]:
wine_table_df.printSchema()

root
 |-- country: string (nullable = true)
 |-- points: string (nullable = true)
 |-- province: string (nullable = true)
 |-- region_1: string (nullable = true)
 |-- region_2: string (nullable = true)
 |-- title: string (nullable = true)
 |-- variety: string (nullable = true)
 |-- winery: string (nullable = true)



In [17]:
#Change "points" column datatype from string to integer
from pyspark.sql.types import IntegerType
wine_table_df = wine_table_df.withColumn("points", wine_table_df["points"].cast(IntegerType()))

In [18]:
wine_table_df.printSchema()

root
 |-- country: string (nullable = true)
 |-- points: integer (nullable = true)
 |-- province: string (nullable = true)
 |-- region_1: string (nullable = true)
 |-- region_2: string (nullable = true)
 |-- title: string (nullable = true)
 |-- variety: string (nullable = true)
 |-- winery: string (nullable = true)



In [19]:
wine_table_df.count()

129912

In [21]:
weather_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Winery: string (nullable = true)
 |-- Average Temperature (Kelvin): double (nullable = true)
 |-- Average Air Pressure (hPa): double (nullable = true)
 |-- Average Humidity (%): double (nullable = true)
 |-- Average Precipitation (mm): double (nullable = true)



In [27]:
#Select only the columns that will be stored into the RDS database

from pyspark.sql.functions import col

weather_table_df = weather_df.select(col("Winery").alias("winery"), col("Average Temperature (Kelvin)").alias("avg_temp"), col("Average Air Pressure (hPa)").alias("avg_airp"), col("Average Humidity (%)").alias("avg_humid"), col("Average Precipitation (mm)").alias("avg_precip"))
weather_table_df.show()

+--------------------+--------+---------+---------+----------+
|              winery|avg_temp| avg_airp|avg_humid|avg_precip|
+--------------------+--------+---------+---------+----------+
|St. Julian winery US|269.6375|  1018.15|    79.24|    0.0725|
|Sweet Cheeks wine...| 276.025|1019.8875|  85.4525|    0.0575|
|Kirkland Signatur...| 277.375|1020.0125|  79.1975|     0.235|
|Louis M. Martini ...|281.0325|   1018.5|   79.155|    0.0775|
|Richard Böcking w...|  275.71|1014.9425|   92.215|     0.135|
|   Bianchi winery US| 279.685|  1018.23|    73.42|     0.025|
|Castello di Amoro...|281.0325|   1018.5|   79.155|    0.0775|
|   Envolve winery US|281.7425|1019.1525|  70.4875|     0.125|
|     Erath winery US|  276.22|1020.7575|   76.705|      0.14|
|Feudi del Pisciot...| 282.535|  1014.39|  75.9475|     0.055|
|Hawkins Cellars w...|276.0775|1020.8975|  74.8725|     0.135|
|Robert Hall winer...| 279.685|  1018.23|    73.42|     0.025|
|Sundance winery C...| 269.755|  1013.96|     58.3|    

In [28]:
weather_table_df.count()

4534

**Connect to the AWS RDS instance and write each DataFrame to its table**

In [29]:
# Configure settings for RDS
mode = "append"
#jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
jdbc_url="jdbc:postgresql://wine-final-project.czqkltznl3rl.us-east-2.rds.amazonaws.com/winedb"
#config = {"user":"postgres",
          #"password": "<password>",
          #"driver":"org.postgresql.Driver"}
config = {"user":"wineuser",
          "password": rds_pwd,
          "driver":"org.postgresql.Driver"}

In [24]:
wine_table_df

DataFrame[country: string, points: int, province: string, region_1: string, region_2: string, title: string, variety: string, winery: string]

In [25]:
# Write wine_table_df to table in RDS
wine_table_df.write.jdbc(url=jdbc_url, table='wine', mode=mode, properties=config)

In [30]:
# Write weather_table_df to table in RDS
weather_table_df.write.jdbc(url=jdbc_url, table='weather', mode=mode, properties=config)

In [ ]:
# Read wine table from RDS
spark.read.jdbc(url=jdbc_url, table='wine', properties=config).limit(10).show()

+------------+------+------------+--------------------+---------------+--------------------+--------------------+-------------------+
|     country|points|    province|            region_1|       region_2|               title|             variety|             winery|
+------------+------+------------+--------------------+---------------+--------------------+--------------------+-------------------+
|   Australia|    90|    Victoria|        Yarra Valley|           null|Giant Steps 2006 ...|          Chardonnay|        Giant Steps|
|          US|    90|  Washington|        Red Mountain|Columbia Valley|Gorman 2006 The P...|               Syrah|             Gorman|
|          US|    90|  Washington|Columbia Valley (WA)|Columbia Valley|Guardian 2006 Cha...|           Red Blend|           Guardian|
|          US|    90|  Washington|        Red Mountain|Columbia Valley|Barrister 2005 Ar...|              Merlot|          Barrister|
|      France|    90|    Bordeaux|        Saint-Julien|       

In [31]:
# Read weather table from RDS
spark.read.jdbc(url=jdbc_url, table='weather', properties=config).limit(10).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|              winery|            avg_temp|            avg_airp|           avg_humid|          avg_precip|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|St. Julian winery US|269.6375000000000...|1018.150000000000...|79.24000000000000...|0.072500000000000000|
|Sweet Cheeks wine...|276.0250000000000...|1019.887500000000...|85.45250000000000...|0.057500000000000000|
|Kirkland Signatur...|277.3750000000000...|1020.012500000000...|79.19750000000000...|0.235000000000000000|
|Louis M. Martini ...|281.0325000000000...|1018.500000000000...|79.15500000000000...|0.077500000000000000|
|Richard Böcking w...|275.7100000000000...|1014.942500000000...|92.21500000000000...|0.135000000000000000|
|   Bianchi winery US|279.6850000000000...|1018.230000000000...|73.42000000000000...|0.025000000000000000|
|Castello di Amoro...|281.03250000000